<a href="https://colab.research.google.com/github/AAA123404/Financial-big-data-analysis/blob/main/newsbert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
import numpy as np
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
tokenizer = AutoTokenizer.from_pretrained("flax-community/roberta-swahili-news-classification")
model = AutoModelForSequenceClassification.from_pretrained("flax-community/roberta-swahili-news-classification")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/642k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/422M [00:00<?, ?B/s]

In [ ]:
model

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(25165, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

In [ ]:
import torch.nn as nn

In [ ]:
train = pd.read_csv("/content/drive/MyDrive/自然語言/Train.csv")
test = pd.read_csv("/content/drive/MyDrive/自然語言/Test.csv")
ss =   pd.read_csv("/content/drive/MyDrive/自然語言/SampleSubmission.csv")

In [ ]:
train['content'] = train['content'].str.lower()
train['category'] = train['category'].str.lower()
test['content'] = test['content'].str.lower()

In [ ]:
train

,id,content,category
0,SW0,serikali imesema haitakuwa tayari kuona amani...,kitaifa
1,SW1,"mkuu wa mkoa wa tabora, aggrey mwanri amesiti...",biashara
2,SW10,serikali imetoa miezi sita kwa taasisi zote z...,kitaifa
3,SW100,kampuni ya mchezo wa kubahatisha ya m-bet ime...,michezo
4,SW1000,watanzania wamekumbushwa kusherehekea sikukuu...,kitaifa
...,...,...,...
5146,SW993,rais john magufuli ameendelea kung’ara katika...,kitaifa
5147,SW994,kampeni inayohimiza watafi ti kuandika upya h...,kitaifa
5148,SW996,"matukio mapya ya malaria kwa kila watu 1,000 ...",kitaifa
5149,SW997,imeelezwa kuwa hakuna sheria yoyote inayokata...,kitaifa


In [ ]:
model

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(25165, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

In [ ]:
train.iloc[1]

id                                                        SW1
content      mkuu wa mkoa wa tabora, aggrey mwanri amesiti...
category                                             biashara
Name: 1, dtype: object

In [ ]:
from torch.utils.data import Dataset, DataLoader

class NewsDataset(Dataset):
    def __init__(self, data, tokenizer, max_len, label_mapping):
        self.data = data
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.label_mapping = label_mapping

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
      row = self.data.iloc[index]
      text = row['content']
      encodings = self.tokenizer(text, max_length=self.max_len, padding='max_length', truncation=True, return_tensors='pt')
      input_ids = encodings['input_ids'].squeeze()
      label = self.label_mapping[row['category']]
      return input_ids, label


train_data = train
label_mapping = {"kitaifa": 0, "michezo": 1, "biashara":2  , "kimataifa":3 , "burudani":4 } # 根據你的類別構建映射 kitaifa	michezo	biashara	kimataifa	burudani
dataset = NewsDataset(train_data, tokenizer, max_len=512 , label_mapping=label_mapping)
dataloader = DataLoader(dataset, batch_size=8, shuffle=True)

In [ ]:
from tqdm import tqdm

def train(model, dataloader, optimizer, device):
    model.train()
    total_loss = 0
    for batch in tqdm(dataloader, total=len(dataloader)):
        input_ids, labels = batch
        input_ids = input_ids.squeeze(1).to(device)
        labels = labels.to(device)
        outputs = model(input_ids, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    return total_loss / len(dataloader)

# 设置设备和优化器
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)

In [ ]:
num_epochs = 10
for epoch in range(num_epochs):
    loss = train(model, dataloader, optimizer, device)
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss}')

100%|██████████| 644/644 [08:11<00:00,  1.31it/s]


Epoch 1/10, Loss: 0.38830376221214113


100%|██████████| 644/644 [08:09<00:00,  1.31it/s]


Epoch 2/10, Loss: 0.21431634980774492


100%|██████████| 644/644 [08:10<00:00,  1.31it/s]


...

Epoch 8/10, Loss: 0.04208740867391086


100%|██████████| 644/644 [08:11<00:00,  1.31it/s]


Epoch 9/10, Loss: 0.03886554677325723


100%|██████████| 644/644 [08:11<00:00,  1.31it/s]

Epoch 10/10, Loss: 0.04301029921134049


In [ ]:
import torch.nn as nn
num_labels = 5
model.classifier.out_proj = nn.Linear(in_features=768, out_features=5, bias=True)

In [ ]:
model

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(25165, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

In [ ]:
class TestDataset(torch.utils.data.Dataset):
    def __init__(self, data, tokenizer, max_len):
        self.data = data
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        row = self.data.iloc[index]
        text = row['content']
        encodings = self.tokenizer(text, max_length=self.max_len, padding='max_length', truncation=True, return_tensors='pt')
        input_ids = encodings['input_ids'].squeeze().to(device)
        return input_ids


test_data = test
testdataset = TestDataset(test_data, tokenizer, max_len=512)
testdataloader = DataLoader(testdataset, batch_size=8, shuffle=True)


In [ ]:
import torch.nn.functional as F
model.eval()
all_probs = []
model.to(device)

for batch in testdataloader:
    input_ids = batch
    with torch.no_grad():
        outputs = model(input_ids=input_ids)
        logits = outputs.logits
        probs = F.softmax(logits, dim=1)
        all_probs.append(probs.cpu().numpy())

all_probs = np.concatenate(all_probs, axis=0)

In [ ]:
all_probs[1]

array([0.43998975, 0.1243673 , 0.12686402, 0.21183881, 0.09694009],
      dtype=float32)

In [ ]:
all_probs.shape

(1288, 5)

In [ ]:
sub_df = pd.DataFrame(all_probs, columns= ['biashara', 'burudani', 'kimataifa', 'kitaifa', 'michezo'])  # generate dataframe to store results
sub_df['swahili_id'] = test['swahili_id']

sub_df = sub_df[['swahili_id','kitaifa','michezo','biashara','kimataifa','burudani']]


In [ ]:
sub_df.head()

,swahili_id,kitaifa,michezo,biashara,kimataifa,burudani
0,001dd47ac202d9db6624a5ff734a5e7dddafeaf2,0.242448,0.120757,0.409060,0.125536,0.102200
1,0043d97f7690e9bc02f0ed8bb2b260d1d44bad92,0.211839,0.096940,0.439990,0.126864,0.124367
2,00579c2307b5c11003d21c40c3ecff5e922c3fd8,0.218315,0.124996,0.381796,0.152380,0.122513
3,00868eeee349e286303706ef0ffd851f39708d37,0.217858,0.179873,0.143668,0.123631,0.334970
4,00a5cb12d3058dcf2e42f277eee599992db32412,0.280353,0.109242,0.374195,0.138713,0.097497


In [ ]:
SUB_FILE_NAME = 'submission_ro.csv'

In [ ]:
sub_df.to_csv(SUB_FILE_NAME, index=False)
from google.colab import files
files.download(SUB_FILE_NAME)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>